In [ ]:
!uv pip install datasets

Using Python 3.12.12 environment at: /usr
Audited 1 package in 242ms


In [ ]:
"""
The system trains BERT (or any other transformer model like RoBERTa, DistilBERT etc.) on the SNLI + MultiNLI (AllNLI) dataset
with MatryoshkaLoss using MultipleNegativesRankingLoss. This trains a model at output dimensions [768, 512, 256, 128, 64].
Entailments are positive pairs and the contradiction on AllNLI dataset is added as a hard negative.
At every 10% training steps, the model is evaluated on the STS benchmark dataset
"""

import logging
import sys
import traceback
from datetime import datetime

from datasets import load_dataset

from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    losses,
)
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction, SequentialEvaluator,TripletEvaluator
from sentence_transformers.training_args import BatchSamplers

In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `full` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `full`


---ALL-NLI-TR Training Phase---


In [ ]:
model_name = 'jhu-clsp/mmBERT-small'
model = SentenceTransformer(model_name, trust_remote_code=True)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/564M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Performance Evaluation before fine-tuning

In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

README.md:   0%|          | 0.00/566 [00:00<?, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

dev.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

...

test/tr.jsonl.gz:   0%|          | 0.00/328k [00:00<?, ?B/s]

train/tr.jsonl.gz:   0%|          | 0.00/560k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/208 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/390 [00:00<?, ? examples/s]

{'sts22-test_pearson_cosine': -0.14583900485477957, 'sts22-test_spearman_cosine': 0.29288193849758515}
{'all-nli-tr-test_cosine_accuracy': 0.65477454662323}


All-nli-tr Fine-tuning phase

In [ ]:
# Set the log level to INFO to get more information
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)


batch_size = 64  # The larger you select this, the better the results (usually). But it requires more GPU memory
num_train_epochs = 1

# Save path of the model
output_dir = f"output/matryoshka_nli_{model_name.replace('/', '-')}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"


# If we want, we can limit the maximum sequence length for the model
model.max_seq_length = 512
logging.info(model)

# 2. Load the AllNLI dataset: https://huggingface.co/datasets/sentence-transformers/all-nli
train_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="train")#select(range(5000))
eval_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="dev")#select(range(1000))
test_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="test")#select(range(1000))
logging.info(train_dataset)

# 3. Define our training loss
inner_train_loss = losses.MultipleNegativesRankingLoss(model)
train_loss = losses.MatryoshkaLoss(model, inner_train_loss, [384, 256, 128, 64]) #Modelin dimention'larına göre ayarlanabilir

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
stsb_eval_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="validation")
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=stsb_eval_dataset["sentence1"],
    sentences2=stsb_eval_dataset["sentence2"],
    scores=stsb_eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-dev-{model_name.replace('/', '-')}",
)


# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    learning_rate=1e-5,
    warmup_steps=144,
    weight_decay=0.01,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    logging_steps=1000,
    run_name=f"matryoshka-allnli-{model_name.replace('/', '-')}",  # Will be used in W&B if `wandb` is installed
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()

# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["sentence1"],
    sentences2=test_dataset["sentence2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-test-{model_name.replace('/', '-')}",
)
test_evaluator(model)

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ozayezerceli (nlp_wizards) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Sts-dev-jhu-clsp-mmbert-small Pearson Cosine,Sts-dev-jhu-clsp-mmbert-small Spearman Cosine
1000,5.172600,3.022047,0.789168,0.792294
2000,3.114200,2.595331,0.806103,0.808717
3000,2.712400,2.416250,0.807078,0.810280
4000,2.518800,2.312061,0.809280,0.811045
5000,2.362500,2.242876,0.809980,0.812003
6000,2.253800,2.189405,0.811333,0.814453
7000,2.159000,2.145348,0.809626,0.812767


Evaluation after fine-tuning

In [ ]:
!hf auth logout

Successfully logged out from all access tokens.


In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `full` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `full`


In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.7774732773366015, 'stsb-mt-turkish-test_spearman_cosine': 0.7847084199775788}
{'sts22-crosslingual-sts-test_pearson_cosine': 0.4123652383649655, 'sts22-crosslingual-sts-test_spearman_cosine': 0.46936193327067}
{'all-nli-tr-test_cosine_accuracy': 0.9259147047996521}


STSB-TR Fine-tuning Phase

In [ ]:
# 3. Define our training loss
# CoSENTLoss (https://sbert.net/docs/package_reference/sentence_transformer/losses.html#cosentloss) needs two text columns and one
# similarity score column (between 0 and 1)

logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)

batch_size = 256
num_train_epochs = 5
matryoshka_dims = [384, 256, 128, 64]

# Save path of the model
output_dir = f"output/matryoshka_stsb_{model_name.replace('/', '-')}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

logging.info(model)

train_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="train")
eval_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="validation")
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")

logging.info(train_dataset)

inner_train_loss = losses.CoSENTLoss(model=model)
train_loss = losses.MatryoshkaLoss(model, loss=inner_train_loss, matryoshka_dims=matryoshka_dims)

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
evaluators = []
for dim in matryoshka_dims:
    evaluators.append(
        EmbeddingSimilarityEvaluator(
            sentences1=eval_dataset["sentence1"],
            sentences2=eval_dataset["sentence2"],
            scores=eval_dataset["score"],
            main_similarity=SimilarityFunction.COSINE,
            name=f"stsb-dev-{dim}",
            truncate_dim=dim,
        )
    )
dev_evaluator = SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[0])

# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    learning_rate=1e-5,
    warmup_steps=144,
    weight_decay=0.01,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    logging_steps=200,
    run_name=f"matryoshka-stsb-{model_name.replace('/', '-')}",  # Will be used in W&B if `wandb` is installed
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()


# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
evaluators = []
for dim in matryoshka_dims:
    evaluators.append(
        EmbeddingSimilarityEvaluator(
            sentences1=test_dataset["sentence1"],
            sentences2=test_dataset["sentence2"],
            scores=test_dataset["score"],
            main_similarity=SimilarityFunction.COSINE,
            name=f"stsb-test-{dim}",
            truncate_dim=dim,
        )
    )
test_evaluator = SequentialEvaluator(evaluators)
test_evaluator(model)

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)

Step,Training Loss,Validation Loss,Stsb-dev-384 Pearson Cosine,Stsb-dev-384 Spearman Cosine,Stsb-dev-256 Pearson Cosine,Stsb-dev-256 Spearman Cosine,Stsb-dev-128 Pearson Cosine,Stsb-dev-128 Spearman Cosine,Stsb-dev-64 Pearson Cosine,Stsb-dev-64 Spearman Cosine,Sequential Score
200,27.268000,23.938068,0.819682,0.843603,0.821038,0.845398,0.810003,0.841039,0.792778,0.830832,0.843603
400,22.092600,25.089687,0.835765,0.848059,0.834931,0.848182,0.826938,0.845643,0.812797,0.837144,0.848059
600,20.767300,27.344318,0.839953,0.847724,0.839279,0.848105,0.832041,0.845544,0.818823,0.837156,0.847724
800,19.689800,29.542440,0.845975,0.852779,0.843876,0.852091,0.835928,0.849660,0.820321,0.838557,0.852779
1000,18.853200,32.115425,0.842580,0.849511,0.840708,0.849014,0.832507,0.846204,0.819027,0.836435,0.849511
1200,18.112400,34.235256,0.846120,0.851996,0.843790,0.850926,0.836559,0.848203,0.823081,0.838373,0.851996
1400,17.413500,36.197361,0.844807,0.850363,0.842220,0.848588,0.833933,0.844592,0.820030,0.834968,0.850363
1600,16.924700,37.159046,0.845158,0.849721,0.842631,0.847723,0.835574,0.844993,0.822462,0.835462,0.849721
1800,16.441000,37.947567,0.845889,0.849847,0.843214,0.847753,0.836455,0.844823,0.823489,0.835184,0.849847


Evaluating after fine-tuning on STSB

In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.8094358855076182, 'stsb-mt-turkish-test_spearman_cosine': 0.8153889845953315}
{'sts22-crosslingual-sts-test_pearson_cosine': 0.47835655549853195, 'sts22-crosslingual-sts-test_spearman_cosine': 0.5204413586368193}
{'all-nli-tr-test_cosine_accuracy': 0.8938819169998169}


In [ ]:
# Upload the model to Hugging Face
model.push_to_hub("mmBERT-small-TR")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/562M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

'https://huggingface.co/ozayezerceli/mmBERT-small-TR/commit/c710d87504b0d77e1312b915f63ef335471297ac'

# **Another Model**

In [ ]:
model_name = 'jhu-clsp/mmBERT-base'
model = SentenceTransformer(model_name, trust_remote_code=True)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.5404137470528421, 'stsb-mt-turkish-test_spearman_cosine': 0.5655373989820383}
{'sts22-crosslingual-sts-test_pearson_cosine': -0.027866533530432044, 'sts22-crosslingual-sts-test_spearman_cosine': 0.34357524440408027}
{'all-nli-tr-test_cosine_accuracy': 0.6919690370559692}


In [ ]:
# Set the log level to INFO to get more information
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)


batch_size = 64  # The larger you select this, the better the results (usually). But it requires more GPU memory
num_train_epochs = 1

# Save path of the model
output_dir = f"output/matryoshka_nli_{model_name.replace('/', '-')}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"


# If we want, we can limit the maximum sequence length for the model
model.max_seq_length = 512
logging.info(model)

# 2. Load the AllNLI dataset: https://huggingface.co/datasets/sentence-transformers/all-nli
train_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="train")#select(range(5000))
eval_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="dev")#select(range(1000))
test_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="test")#select(range(1000))
logging.info(train_dataset)

# 3. Define our training loss
inner_train_loss = losses.MultipleNegativesRankingLoss(model)
train_loss = losses.MatryoshkaLoss(model, inner_train_loss, [768, 512, 384, 256, 128, 64]) #Modelin dimention'larına göre ayarlanabilir

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
stsb_eval_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="validation")
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=stsb_eval_dataset["sentence1"],
    sentences2=stsb_eval_dataset["sentence2"],
    scores=stsb_eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-dev-{model_name.replace('/', '-')}",
)


# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    learning_rate=1e-5,
    warmup_steps=144,
    weight_decay=0.01,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    logging_steps=1000,
    run_name=f"matryoshka-allnli-{model_name.replace('/', '-')}",  # Will be used in W&B if `wandb` is installed
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()

# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["sentence1"],
    sentences2=test_dataset["sentence2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-test-{model_name.replace('/', '-')}",
)
test_evaluator(model)

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Sts-dev-jhu-clsp-mmbert-base Pearson Cosine,Sts-dev-jhu-clsp-mmbert-base Spearman Cosine
1000,6.067700,3.628175,0.824680,0.825761
2000,3.813400,3.152514,0.836829,0.837034
3000,3.328500,2.941200,0.833969,0.834172
4000,3.076200,2.806998,0.834147,0.834869
5000,2.864000,2.738783,0.835977,0.837434
6000,2.756200,2.690998,0.839233,0.840388
7000,2.621900,2.654452,0.837828,0.838669


In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.8096031823370108, 'stsb-mt-turkish-test_spearman_cosine': 0.817889523982502}
{'sts22-crosslingual-sts-test_pearson_cosine': 0.5080123214816751, 'sts22-crosslingual-sts-test_spearman_cosine': 0.5552438747168523}
{'all-nli-tr-test_cosine_accuracy': 0.9400333762168884}


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# 3. Define our training loss
# CoSENTLoss (https://sbert.net/docs/package_reference/sentence_transformer/losses.html#cosentloss) needs two text columns and one
# similarity score column (between 0 and 1)

logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)

batch_size = 128
num_train_epochs = 5
matryoshka_dims = [768, 512, 384, 256, 128, 64]

# Save path of the model
output_dir = f"output/matryoshka_stsb_{model_name.replace('/', '-')}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

logging.info(model)

train_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="train")
eval_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="validation")
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")

logging.info(train_dataset)

inner_train_loss = losses.CoSENTLoss(model=model)
train_loss = losses.MatryoshkaLoss(model, loss=inner_train_loss, matryoshka_dims=matryoshka_dims)

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
evaluators = []
for dim in matryoshka_dims:
    evaluators.append(
        EmbeddingSimilarityEvaluator(
            sentences1=eval_dataset["sentence1"],
            sentences2=eval_dataset["sentence2"],
            scores=eval_dataset["score"],
            main_similarity=SimilarityFunction.COSINE,
            name=f"stsb-dev-{dim}",
            truncate_dim=dim,
        )
    )
dev_evaluator = SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[0])

# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    learning_rate=1e-5,
    warmup_steps=144,
    weight_decay=0.01,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    logging_steps=200,
    run_name=f"matryoshka-stsb-{model_name.replace('/', '-')}",  # Will be used in W&B if `wandb` is installed
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()


# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
evaluators = []
for dim in matryoshka_dims:
    evaluators.append(
        EmbeddingSimilarityEvaluator(
            sentences1=test_dataset["sentence1"],
            sentences2=test_dataset["sentence2"],
            scores=test_dataset["score"],
            main_similarity=SimilarityFunction.COSINE,
            name=f"stsb-test-{dim}",
            truncate_dim=dim,
        )
    )
test_evaluator = SequentialEvaluator(evaluators)
test_evaluator(model)

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)

Step,Training Loss,Validation Loss,Stsb-dev-768 Pearson Cosine,Stsb-dev-768 Spearman Cosine,Stsb-dev-512 Pearson Cosine,Stsb-dev-512 Spearman Cosine,Stsb-dev-384 Pearson Cosine,Stsb-dev-384 Spearman Cosine,Stsb-dev-256 Pearson Cosine,Stsb-dev-256 Spearman Cosine,Stsb-dev-128 Pearson Cosine,Stsb-dev-128 Spearman Cosine,Stsb-dev-64 Pearson Cosine,Stsb-dev-64 Spearman Cosine,Sequential Score
200,56.811400,55.814911,0.851952,0.860005,0.853364,0.860976,0.852063,0.861240,0.844518,0.856717,0.835126,0.851725,0.821165,0.845199,0.860005


In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.8423645332640702, 'stsb-mt-turkish-test_spearman_cosine': 0.8541832122547709}
{'sts22-crosslingual-sts-test_pearson_cosine': 0.5093209716332461, 'sts22-crosslingual-sts-test_spearman_cosine': 0.5793588847714382}
{'all-nli-tr-test_cosine_accuracy': 0.9204493761062622}


In [ ]:
model.push_to_hub("mmBERT-base-TR")

model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/ozayezerceli/mmBERT-base-TR/commit/d2c45c86ca983e13f7f14d74b5046930232d205d'

# **Another Model**

In [ ]:
model_name = 'jhu-clsp/ettin-encoder-32m'
model = SentenceTransformer(model_name, trust_remote_code=True)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/128M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/128M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

W1015 10:28:46.801000 668 torch/_dynamo/convert_frame.py:1016] [0/8] torch._dynamo hit config.recompile_limit (8)
W1015 10:28:46.801000 668 torch/_dynamo/convert_frame.py:1016] [0/8]    function: 'compiled_embeddings' (/usr/local/lib/python3.12/dist-packages/transformers/models/modernbert/modeling_modernbert.py:207)
W1015 10:28:46.801000 668 torch/_dynamo/convert_frame.py:1016] [0/8]    last reason: 0/7: GLOBAL_STATE changed: autocast 
W1015 10:28:46.801000 668 torch/_dynamo/convert_frame.py:1016] [0/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W1015 10:28:46.801000 668 torch/_dynamo/convert_frame.py:1016] [0/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.
W1015 10:28:46.826000 668 torch/_dynamo/convert_frame.py:1016] [1/8] torch._dynamo hit config.recompile_limit (8)
W1015 10:28:46.826000 668 torch/_dynamo/convert_frame.py:1016] [1/8]    function: 'compiled_mlp' (/usr/local/lib/python3.12/dist-packages/tr

{'stsb-mt-turkish-test_pearson_cosine': 0.43085840164895506, 'stsb-mt-turkish-test_spearman_cosine': 0.45654811113877425}
{'sts22-crosslingual-sts-test_pearson_cosine': 0.022743696017443665, 'sts22-crosslingual-sts-test_spearman_cosine': 0.23140735023589734}
{'all-nli-tr-test_cosine_accuracy': 0.6409594416618347}


In [ ]:
# Set the log level to INFO to get more information
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)


batch_size = 64  # The larger you select this, the better the results (usually). But it requires more GPU memory
num_train_epochs = 1

# Save path of the model
output_dir = f"output/matryoshka_nli_{model_name.replace('/', '-')}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"


# If we want, we can limit the maximum sequence length for the model
model.max_seq_length = 512
logging.info(model)

# 2. Load the AllNLI dataset: https://huggingface.co/datasets/sentence-transformers/all-nli
train_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="train")#select(range(5000))
eval_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="dev")#select(range(1000))
test_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="test")#select(range(1000))
logging.info(train_dataset)

# 3. Define our training loss
inner_train_loss = losses.MultipleNegativesRankingLoss(model)
train_loss = losses.MatryoshkaLoss(model, inner_train_loss, [384, 256, 128, 64]) #Modelin dimention'larına göre ayarlanabilir

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
stsb_eval_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="validation")
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=stsb_eval_dataset["sentence1"],
    sentences2=stsb_eval_dataset["sentence2"],
    scores=stsb_eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-dev-{model_name.replace('/', '-')}",
)


# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    learning_rate=1e-5,
    warmup_steps=144,
    weight_decay=0.01,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    logging_steps=1000,
    run_name=f"matryoshka-allnli-{model_name.replace('/', '-')}",  # Will be used in W&B if `wandb` is installed
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()

# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["sentence1"],
    sentences2=test_dataset["sentence2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-test-{model_name.replace('/', '-')}",
)
test_evaluator(model)

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Sts-dev-jhu-clsp-ettin-encoder-32m Pearson Cosine,Sts-dev-jhu-clsp-ettin-encoder-32m Spearman Cosine
1000,8.915900,6.552272,0.732014,0.728214
2000,6.383400,5.728661,0.743632,0.741277
3000,5.652500,5.359614,0.751635,0.749377
4000,5.302800,5.133930,0.754398,0.752762
5000,5.077300,4.978953,0.759018,0.757364
6000,4.933100,4.877614,0.760238,0.758776
7000,4.775100,4.812497,0.761012,0.758939


In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.7229152022302153, 'stsb-mt-turkish-test_spearman_cosine': 0.7161797378453192}
{'sts22-crosslingual-sts-test_pearson_cosine': 0.3579698985062576, 'sts22-crosslingual-sts-test_spearman_cosine': 0.4602708098089001}
{'all-nli-tr-test_cosine_accuracy': 0.836344301700592}


In [ ]:
# 3. Define our training loss
# CoSENTLoss (https://sbert.net/docs/package_reference/sentence_transformer/losses.html#cosentloss) needs two text columns and one
# similarity score column (between 0 and 1)

logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)

batch_size = 256
num_train_epochs = 5
matryoshka_dims = [384, 256, 128, 64]

# Save path of the model
output_dir = f"output/matryoshka_stsb_{model_name.replace('/', '-')}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

logging.info(model)

train_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="train")
eval_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="validation")
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")

logging.info(train_dataset)

inner_train_loss = losses.CoSENTLoss(model=model)
train_loss = losses.MatryoshkaLoss(model, loss=inner_train_loss, matryoshka_dims=matryoshka_dims)

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
evaluators = []
for dim in matryoshka_dims:
    evaluators.append(
        EmbeddingSimilarityEvaluator(
            sentences1=eval_dataset["sentence1"],
            sentences2=eval_dataset["sentence2"],
            scores=eval_dataset["score"],
            main_similarity=SimilarityFunction.COSINE,
            name=f"stsb-dev-{dim}",
            truncate_dim=dim,
        )
    )
dev_evaluator = SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[0])

# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    learning_rate=1e-5,
    warmup_steps=144,
    weight_decay=0.01,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    logging_steps=200,
    run_name=f"matryoshka-stsb-{model_name.replace('/', '-')}",  # Will be used in W&B if `wandb` is installed
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()


# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
evaluators = []
for dim in matryoshka_dims:
    evaluators.append(
        EmbeddingSimilarityEvaluator(
            sentences1=test_dataset["sentence1"],
            sentences2=test_dataset["sentence2"],
            scores=test_dataset["score"],
            main_similarity=SimilarityFunction.COSINE,
            name=f"stsb-test-{dim}",
            truncate_dim=dim,
        )
    )
test_evaluator = SequentialEvaluator(evaluators)
test_evaluator(model)

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)

Step,Training Loss,Validation Loss,Stsb-dev-384 Pearson Cosine,Stsb-dev-384 Spearman Cosine,Stsb-dev-256 Pearson Cosine,Stsb-dev-256 Spearman Cosine,Stsb-dev-128 Pearson Cosine,Stsb-dev-128 Spearman Cosine,Stsb-dev-64 Pearson Cosine,Stsb-dev-64 Spearman Cosine,Sequential Score
20,No log,61.087814,0.756988,0.756107,0.757083,0.756987,0.753049,0.755555,0.723993,0.729901,0.756107
40,No log,46.779114,0.718666,0.729254,0.722760,0.732247,0.711724,0.726567,0.688892,0.711711,0.729254
60,No log,40.511089,0.637064,0.665481,0.650337,0.677188,0.633965,0.667260,0.621033,0.660721,0.665481
80,No log,40.212170,0.724424,0.741334,0.719381,0.739176,0.700023,0.727226,0.676575,0.714179,0.741334
100,No log,40.117359,0.747120,0.761333,0.738223,0.756394,0.721115,0.745118,0.697944,0.731209,0.761333


In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.7244123302030215, 'stsb-mt-turkish-test_spearman_cosine': 0.7402675096909954}
{'sts22-crosslingual-sts-test_pearson_cosine': 0.17869514513977258, 'sts22-crosslingual-sts-test_spearman_cosine': 0.3755620719278341}
{'all-nli-tr-test_cosine_accuracy': 0.8011234402656555}


In [ ]:
model.push_to_hub("ettin-encoder-32M-TR")

model.safetensors:   0%|          | 0.00/128M [00:00<?, ?B/s]

'https://huggingface.co/ozayezerceli/ettin-encoder-32M-TR/commit/d9b7e23fb4840c367fda9a831650ff48c89f6251'

# **Another Model**

In [ ]:
model_name = 'jhu-clsp/ettin-encoder-150m'
model = SentenceTransformer(model_name, trust_remote_code=True)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.337724648651456, 'stsb-mt-turkish-test_spearman_cosine': 0.40595644827559707}
{'sts22-crosslingual-sts-test_pearson_cosine': -0.172002297877412, 'sts22-crosslingual-sts-test_spearman_cosine': 0.17664867591803116}
{'all-nli-tr-test_cosine_accuracy': 0.6178836822509766}


In [ ]:
# Set the log level to INFO to get more information
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)


batch_size = 64  # The larger you select this, the better the results (usually). But it requires more GPU memory
num_train_epochs = 1

# Save path of the model
output_dir = f"output/matryoshka_nli_{model_name.replace('/', '-')}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"


# If we want, we can limit the maximum sequence length for the model
model.max_seq_length = 512
logging.info(model)

# 2. Load the AllNLI dataset: https://huggingface.co/datasets/sentence-transformers/all-nli
train_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="train")#select(range(5000))
eval_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="dev")#select(range(1000))
test_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="test")#select(range(1000))
logging.info(train_dataset)

# 3. Define our training loss
inner_train_loss = losses.MultipleNegativesRankingLoss(model)
train_loss = losses.MatryoshkaLoss(model, inner_train_loss, [768, 512, 384, 256, 128, 64]) #Modelin dimention'larına göre ayarlanabilir

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
stsb_eval_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="validation")
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=stsb_eval_dataset["sentence1"],
    sentences2=stsb_eval_dataset["sentence2"],
    scores=stsb_eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-dev-{model_name.replace('/', '-')}",
)


# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    learning_rate=1e-5,
    warmup_steps=144,
    weight_decay=0.01,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    logging_steps=1000,
    run_name=f"matryoshka-allnli-{model_name.replace('/', '-')}",  # Will be used in W&B if `wandb` is installed
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()

# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["sentence1"],
    sentences2=test_dataset["sentence2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-test-{model_name.replace('/', '-')}",
)
test_evaluator(model)

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Sts-dev-jhu-clsp-ettin-encoder-150m Pearson Cosine,Sts-dev-jhu-clsp-ettin-encoder-150m Spearman Cosine
1000,12.705600,8.804202,0.757612,0.754955
2000,8.665600,7.755711,0.765324,0.763226
3000,7.627300,7.310081,0.769851,0.768073
4000,7.076500,6.935140,0.777303,0.775381
5000,6.694800,6.668308,0.778483,0.776998
6000,6.442500,6.491343,0.779790,0.778408
7000,6.190800,6.412029,0.779556,0.778202


In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.7330295457280083, 'stsb-mt-turkish-test_spearman_cosine': 0.7263168729407058}
{'sts22-crosslingual-sts-test_pearson_cosine': 0.3660247443160787, 'sts22-crosslingual-sts-test_spearman_cosine': 0.43256669136977055}
{'all-nli-tr-test_cosine_accuracy': 0.8501594066619873}


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# 3. Define our training loss
# CoSENTLoss (https://sbert.net/docs/package_reference/sentence_transformer/losses.html#cosentloss) needs two text columns and one
# similarity score column (between 0 and 1)

logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)

batch_size = 128
num_train_epochs = 5
matryoshka_dims = [768, 512, 384, 256, 128, 64]

# Save path of the model
output_dir = f"output/matryoshka_stsb_{model_name.replace('/', '-')}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

logging.info(model)

train_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="train")
eval_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="validation")
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")

logging.info(train_dataset)

inner_train_loss = losses.CoSENTLoss(model=model)
train_loss = losses.MatryoshkaLoss(model, loss=inner_train_loss, matryoshka_dims=matryoshka_dims)

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
evaluators = []
for dim in matryoshka_dims:
    evaluators.append(
        EmbeddingSimilarityEvaluator(
            sentences1=eval_dataset["sentence1"],
            sentences2=eval_dataset["sentence2"],
            scores=eval_dataset["score"],
            main_similarity=SimilarityFunction.COSINE,
            name=f"stsb-dev-{dim}",
            truncate_dim=dim,
        )
    )
dev_evaluator = SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[0])

# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    learning_rate=1e-5,
    warmup_steps=20,
    weight_decay=0.01,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    logging_steps=200,
    run_name=f"matryoshka-stsb-{model_name.replace('/', '-')}",  # Will be used in W&B if `wandb` is installed
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()


# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
evaluators = []
for dim in matryoshka_dims:
    evaluators.append(
        EmbeddingSimilarityEvaluator(
            sentences1=test_dataset["sentence1"],
            sentences2=test_dataset["sentence2"],
            scores=test_dataset["score"],
            main_similarity=SimilarityFunction.COSINE,
            name=f"stsb-test-{dim}",
            truncate_dim=dim,
        )
    )
test_evaluator = SequentialEvaluator(evaluators)
test_evaluator(model)

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)

Step,Training Loss,Validation Loss,Stsb-dev-768 Pearson Cosine,Stsb-dev-768 Spearman Cosine,Stsb-dev-512 Pearson Cosine,Stsb-dev-512 Spearman Cosine,Stsb-dev-384 Pearson Cosine,Stsb-dev-384 Spearman Cosine,Stsb-dev-256 Pearson Cosine,Stsb-dev-256 Spearman Cosine,Stsb-dev-128 Pearson Cosine,Stsb-dev-128 Spearman Cosine,Stsb-dev-64 Pearson Cosine,Stsb-dev-64 Spearman Cosine,Sequential Score
20,No log,83.056458,0.775949,0.775166,0.772997,0.773078,0.772651,0.772894,0.770250,0.770119,0.759977,0.763871,0.741455,0.748474,0.775166
40,No log,64.305305,0.756982,0.758812,0.754192,0.756909,0.754783,0.758261,0.748578,0.751489,0.735271,0.743540,0.716143,0.727973,0.758812
60,No log,52.211327,0.726180,0.738402,0.723927,0.737927,0.722298,0.739575,0.717071,0.734980,0.703146,0.726862,0.692388,0.719163,0.738402
80,No log,51.760811,0.734021,0.752249,0.731752,0.751662,0.725604,0.750521,0.731421,0.752504,0.709288,0.741318,0.696803,0.733684,0.752249
100,No log,51.779072,0.766272,0.778872,0.765292,0.778832,0.761062,0.778013,0.761527,0.776244,0.738687,0.765001,0.724159,0.755773,0.778872
120,No log,51.379887,0.779705,0.787446,0.778657,0.787253,0.776861,0.787566,0.773651,0.783633,0.756115,0.774779,0.742589,0.765092,0.787446
140,No log,51.362949,0.784864,0.790191,0.784513,0.790560,0.783260,0.791359,0.779334,0.786520,0.762822,0.778030,0.748569,0.768334,0.790191
160,No log,51.513062,0.781945,0.789673,0.781464,0.789939,0.778805,0.790256,0.776910,0.786377,0.758615,0.777128,0.742450,0.766648,0.789673
180,No log,51.786541,0.782884,0.792054,0.781930,0.791784,0.779016,0.791561,0.777520,0.788500,0.757027,0.778838,0.740404,0.768083,0.792054
200,56.719500,51.874535,0.785468,0.792624,0.784965,0.792734,0.782263,0.792355,0.778920,0.787733,0.761031,0.778487,0.745606,0.767720,0.792624


In [ ]:
#model_name = 'BAAI/bge-m3'
#model = SentenceTransformer(model_name, trust_remote_code=True)

test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

{'stsb-mt-turkish-test_pearson_cosine': 0.7534270261021556, 'stsb-mt-turkish-test_spearman_cosine': 0.7583372947090957}
{'sts22-crosslingual-sts-test_pearson_cosine': 0.21212254723545598, 'sts22-crosslingual-sts-test_spearman_cosine': 0.3263790389519844}
{'all-nli-tr-test_cosine_accuracy': 0.8322453498840332}


In [ ]:
model.push_to_hub("ettin-encoder-150M-TR")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...szlh01v/model.safetensors:   1%|          | 4.97MB /  596MB            

'https://huggingface.co/ozayezerceli/ettin-encoder-150M-TR/commit/8ccc70fe376d78e20f74cc5002b7affd9fae8669'

# **Another Model**

In [ ]:
model_name = 'jhu-clsp/ettin-encoder-1b'
model = SentenceTransformer(model_name, trust_remote_code=True)

config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.75G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False, 'architecture': 'MistralModel'})
  (1): Pooling({'word_embedding_dimension': 3072, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
test_dataset_stsb = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator_stsb = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_stsb["sentence1"]),
    sentences2=list(test_dataset_stsb["sentence2"]),
    scores=list(test_dataset_stsb["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="stsb-mt-turkish-test",
)

print(test_evaluator_stsb(model))

test_dataset_sts22 = load_dataset("mteb/sts22-crosslingual-sts","tr",split="test")
test_evaluator_sts22 = EmbeddingSimilarityEvaluator(
    sentences1=list(test_dataset_sts22["sentence1"]),
    sentences2=list(test_dataset_sts22["sentence2"]),
    scores=list(test_dataset_sts22["score"]),
    main_similarity=SimilarityFunction.COSINE,
    name="sts22-crosslingual-sts-test",
)

print(test_evaluator_sts22(model))

eval_dataset_nli = load_dataset("emrecan/all-nli-tr", "triplet", split="test")

# Initialize the evaluator
dev_evaluator_nli = TripletEvaluator(
    anchors=list(eval_dataset_nli["anchor"]),
    positives=list(eval_dataset_nli["positive"]),
    negatives=list(eval_dataset_nli["negative"]),
    main_similarity_function=SimilarityFunction.COSINE,
    name=f"all-nli-tr-test",
)

print(dev_evaluator_nli(model))

README.md:   0%|          | 0.00/564 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/519k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/153k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/116k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

...

test.tsv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/482091 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/6567 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6587 [00:00<?, ? examples/s]

{'all-nli-tr-test_cosine_accuracy': 0.6602398753166199}


In [ ]:
# Set the log level to INFO to get more information
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)


batch_size = 64  # The larger you select this, the better the results (usually). But it requires more GPU memory
num_train_epochs = 1

# Save path of the model
output_dir = f"output/matryoshka_nli_{model_name.replace('/', '-')}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"


# If we want, we can limit the maximum sequence length for the model
model.max_seq_length = 512
logging.info(model)

# 2. Load the AllNLI dataset: https://huggingface.co/datasets/sentence-transformers/all-nli
train_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="train")#select(range(5000))
eval_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="dev")#select(range(1000))
test_dataset = load_dataset("emrecan/all-nli-tr", "triplet", split="test")#select(range(1000))
logging.info(train_dataset)

# 3. Define our training loss
inner_train_loss = losses.MultipleNegativesRankingLoss(model)
train_loss = losses.MatryoshkaLoss(model, inner_train_loss, [1792, 1024, 768, 512, 384, 256]) #Modelin dimention'larına göre ayarlanabilir

# 4. Define an evaluator for use during training. This is useful to keep track of alongside the evaluation loss.
stsb_eval_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="validation")
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=stsb_eval_dataset["sentence1"],
    sentences2=stsb_eval_dataset["sentence2"],
    scores=stsb_eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-dev-{model_name.replace('/', '-')}",
)


# 5. Define the training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    learning_rate=1e-5,
    warmup_steps=144,
    weight_decay=0.01,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    logging_steps=1000,
    run_name=f"matryoshka-allnli-{model_name.replace('/', '-')}",  # Will be used in W&B if `wandb` is installed
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)
trainer.train()

# 7. Evaluate the model performance on the STS Benchmark test dataset
test_dataset = load_dataset("ozayezerceli/stsb-deepl-tr", split="test")
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["sentence1"],
    sentences2=test_dataset["sentence2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name=f"sts-test-{model_name.replace('/', '-')}",
)
test_evaluator(model)

# 8. Save the trained & evaluated model locally
final_output_dir = f"{output_dir}/final"
model.save(final_output_dir)